# IMMC 2020
### Importing Modules and Data

In [119]:
import pandas as pd
import copy
import re
import random
import math, statistics
import gc
import matplotlib.pyplot as plt
from pprint import pprint

median_income = 9733/365
loss_aversion_coefficient = 2
max_shelf_capacity = 100

pdt_csv_data = pd.read_csv("StoreData_IMMC_CSV.csv")
print(pdt_csv_data.head())

          department product_category       product_type     brand  \
0    TV&Home Theater   TVS 30" to 45"    1080p LED Smart  Brand BB   
1  Computers&Tablets          Laptops  2-in-1 Chromebook   Brand A   
2  Computers&Tablets          Laptops  2-in-1 Chromebook  Brand BB   
3  Computers&Tablets          Laptops  2-in-1 Chromebook   Brand G   
4  Computers&Tablets          Laptops  2-in-1 Chromebook   Brand P   

                                                name  initial_price  \
0                 40" 1080p Smart LED HDTV, 5 Series         259.99   
1  2-in-1 11.6" Touch-Screen Chromebook, Intel Ce...         279.99   
2  2-in-1 12.2" Touch-Screen Chromebook, Intel Ce...         449.00   
3  2-in-1 14" Touch-Screen Chromebook, Intel Core...         549.00   
4  2-in-1 14" Touch-Screen Chromebook, Intel Core...         599.00   

   discounted_price  qty  customer_rating  brand_rating  percentage_usage  \
0            179.99    8              4.7      4.623810              0.95  

In [120]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

## Determining the Popularity of Product
### Impact of Discount on Popularity

In [121]:
# Traditional Econs Approach
def q1_over_q0(p0, p1, percentage_usage):
    x = (p0 - p1)*(percentage_usage/median_income) 
    return (math.exp(x))

# Behavioural Econs Approach
def prospect_utility(x):
    if x > 0:
        return(math.log(x+1))
    else:
        return(-loss_aversion_coefficient * math.log(-x + 1))

def increase_utility(p0, p1):
    return(prospect_utility(p0 - p1))

# Helper Variables
max_increase_utility = increase_utility(3329.99, 2199.99)
min_increase_utility = 0
max_q1_over_q0 = q1_over_q0(3329.99, 2199.99, 1)
min_q1_over_qo = 1

# Combined Effect [Between 0 and 1]
def popularity_due_to_discount(p0, p1, percentage_usage):
    traditional_econs_adjusted = (q1_over_q0(p0, p1, percentage_usage)-min_q1_over_qo)/max_q1_over_q0
    behavioural_econs_adjusted = (increase_utility(p0, p1)-min_increase_utility)/max_increase_utility

    total_adjusted = statistics.mean([traditional_econs_adjusted, behavioural_econs_adjusted])
    return(total_adjusted)

### Effect of Loss Adversion on Popularity

In [122]:
# [Between 0 and 1]
def popularity_due_to_loss_aversion(qty):
    return(math.exp(-qty/loss_aversion_coefficient))

### Effects of Saliency Bias on Popularity (TO DO)

In [123]:
def popularity_due_to_saliency_bias(size, qty):
    return sigmoid(size*qty)

### Effects of Ratings on Popularity

In [124]:
# [Between 0 and 1]
def popularity_due_to_rating(pdt_rating, brand_rating):
    raw_brand = (0.8*pdt_rating + 0.2*brand_rating)
    return(raw_brand/5)

## Creating Product Class and List

In [125]:
pdt_list = []
class product:
    # Popularity Coefficients
    pop_loss_adversion_coefficient = 0.33
    pop_saliency_coefficient = 0.33
    pop_rating_coefficient = 0.33
    
    
    # Raw Data
    def __init__(self, index, name, department, product_category, product_type, brand, initial_price, discounted_price, qty, customer_rating, brand_rating=5, percentage_usage=0.5, size=20):
        self.name = name
        self.index = index
        self.department = department
        self.product_category = product_category
        self.product_type = product_type
        self.brand = brand
        self.initial_price = initial_price
        self.discounted_price = discounted_price
        self.qty = qty
        self.customer_rating = customer_rating
        self.brand_rating = brand_rating
        self.percentage_usage = percentage_usage
        self.size = size
        
        self.popularity = 0
    
    # Processed Data
    
    def set_popularity(self):
        discount_factor = popularity_due_to_discount(self.initial_price, self.discounted_price, self.percentage_usage)
        loss_adversion_factor = popularity_due_to_loss_aversion(self.qty)
        saliency_factor = popularity_due_to_saliency_bias(self.size, self.qty)
        rating_factor = popularity_due_to_rating(self.customer_rating, self.brand_rating)
        
        initial_popularity = self.pop_loss_adversion_coefficient*loss_adversion_factor + self.pop_saliency_coefficient*saliency_factor + self.pop_rating_coefficient*rating_factor
        self.popularity = sigmoid(initial_popularity + discount_factor)
        
        
        

In [126]:
# Populating the Product List
for index, row in pdt_csv_data.iterrows():
    
    total_qty = row["qty"]
    unit_size = row["index_size"] # TO CHANGE
    while True:
        cur_qty = min(int(max_shelf_capacity/unit_size), total_qty)
        total_qty -= cur_qty
        
        #print(cur_qty)
        
        cur_pdt = product(index, row["name"], row["department"], row["product_category"], row["product_type"], row["brand"], row["initial_price"], row["discounted_price"], cur_qty, row["customer_rating"], size=unit_size, brand_rating=row["brand_rating"], percentage_usage=row["percentage_usage"])
        cur_pdt.set_popularity()
        
        if total_qty <=0:
            break
    
    pdt_list.append(cur_pdt)


In [127]:
pdt_list[4].index

4

## Creating Shelf Class and Layout Object

In [128]:
class shelf:
    # Class Variables
    max_capacity = 100
    
    def __init__(self, department):
        self.pdts = []
        self.pdt_set = set()
        self.department = department
        self.cur_capacity = 0
    
    def add_pdt(self, pdt):
        # Check Department
        print("product_department", pdt.department)
        print("shelf_department", self.department)
        if pdt.department != self.department:
            return -1
        
        # If shelf can accomodate the product
        if self.cur_capacity + pdt.size <= self.max_capacity:
            self.cur_capacity += pdt.size
            self.pdts.append(pdt)
            self.pdt_set.add(pdt.index)
            return 0
        
        # If shelf is full
        return -1

In [129]:
class layout:
    
    def __init__(self, grid, counter, entrance, exit, shelf_list=[], pdt_list=[]):
        self.counter = counter
        self.value_of_goods_bought = 0
        self.entrance = entrance
        self.exit = exit
        
        # Grid is a 2d matrix where shelves are 1 indexed
        self.grid = copy.deepcopy(grid)
        self.shelf_list = copy.deepcopy(shelf_list)
        
        # A* Grid is a grid where shelves are labelled as 1
        self.a_star_grid = copy.deepcopy(grid)
        for i in range(len(self.a_star_grid)):
            for j in range(len(self.a_star_grid)):
                if self.a_star_grid[i][j] > 1:
                    self.a_star_grid[i][j] = 1
                    
        # Shopper Density Grid is a grid to record the density of shoppers; shelves have a density of 99
        self.shopper_density_grid = copy.deepcopy(self.a_star_grid)
        for i in range(len(self.shopper_density_grid)):
            for j in range(len(self.shopper_density_grid)):
                if self.shopper_density_grid[i][j] == 1:
                    self.shopper_density_grid[i][j] = -1
                    
        # Price Density Grid is a grid to record to value of products the customers are carrying at particular locations
        self.price_density_grid = copy.deepcopy(grid)
        for i in range(len(self.price_density_grid)):
            for j in range(len(self.price_density_grid)):
                if self.price_density_grid[i][j] > 0:
                    self.price_density_grid[i][j] = 0
                    
                    
        # pdt_list contains the products that exists somewhere within the layout
        self.pdt_list = copy.deepcopy(pdt_list)
        
        
        
    # A_Star Performs a simulation of a person walking within the layout from init to goal
    # A_Star Returns a list of nodes visited on the path
    # Coordinates are written as [y,x] with [0,0] being the upper left hand corner
    def a_star(self, init, goal): 
        grid = copy.deepcopy(self.a_star_grid)
        
        # init = [0, 0]
        # goal = [len(grid) - 1, len(grid[0]) - 1]  # all coordinates are given in format [y,x]
        
        cost = 1

        # the cost map which pushes the path closer to the goal
#         heuristic = [[0 for row in range(len(grid[0]))] for col in range(len(grid))]
#         for i in range(len(grid)):
#             for j in range(len(grid[0])):
#                 heuristic[i][j] = abs(i - goal[0]) + abs(j - goal[1])
#                 if grid[i][j] == 1:
#                     heuristic[i][j] = 99  # added extra penalty in the heuristic map

        heuristic = [[0 for row in range(len(grid[0]))] for col in range(len(grid))]
        for i in range(len(grid)):
            for j in range(len(grid[0])):
                heuristic[i][j] = abs(i - goal[0]) + abs(j - goal[1])
                if grid[i][j] == 1:
                    heuristic[i][j] = 999999  # added extra penalty in the heuristic map
                else:
                    heuristic[i][j] = self.shopper_density_grid[i][j]



        # the actions we can take
        delta = [[-1, 0], [0, -1], [1, 0], [0, 1]]  # go up  # go left  # go down  # go right


        # function to search the path
        def search(grid, init, goal, cost, heuristic):

            closed = [
                [0 for col in range(len(grid[0]))] for row in range(len(grid))
            ]  # the reference grid
            closed[init[0]][init[1]] = 1
            action = [
                [0 for col in range(len(grid[0]))] for row in range(len(grid))
            ]  # the action grid

            x = init[0]
            y = init[1]
            g = 0
            f = g + heuristic[init[0]][init[0]]
            cell = [[f, g, x, y]]

            found = False  # flag that is set when search is complete
            resign = False  # flag set if we can't find expand

            while not found and not resign:
                if len(cell) == 0:
                    return "FAIL"
                else:
                    cell.sort()  # to choose the least costliest action so as to move closer to the goal
                    cell.reverse()
                    next = cell.pop()
                    x = next[2]
                    y = next[3]
                    g = next[1]

                    if x == goal[0] and y == goal[1]:
                        found = True
                    else:
                        for i in range(len(delta)):  # to try out different valid actions
                            x2 = x + delta[i][0]
                            y2 = y + delta[i][1]
                            if x2 >= 0 and x2 < len(grid) and y2 >= 0 and y2 < len(grid[0]):
                                if closed[x2][y2] == 0 and grid[x2][y2] == 0:
                                    g2 = g + cost
                                    f2 = g2 + heuristic[x2][y2]
                                    cell.append([f2, g2, x2, y2])
                                    closed[x2][y2] = 1
                                    action[x2][y2] = i
            invpath = []
            x = goal[0]
            y = goal[1]
            invpath.append([x, y])  # we get the reverse path from here
            while x != init[0] or y != init[1]:
                x2 = x - delta[action[x][y]][0]
                y2 = y - delta[action[x][y]][1]
                x = x2
                y = y2
                invpath.append([x, y])

            path = []
            for i in range(len(invpath)):
                path.append(invpath[len(invpath) - 1 - i])
                
#             print("ACTION MAP")
#             for i in range(len(action)):
#                 print(action[i])

            return path
        return search(grid, init, goal, cost, heuristic)
    
    # Simulates Choice of Object to Buy and Deletes that Object from Object List
    def choose_pdt(self):
        #print("pdt list len", len(self.pdt_list))
        if len(self.pdt_list) == 0:
            print("RAN OUT OF ITEMS IN SHOP")
            return -1
        self.pdt_list.sort(key=lambda x: x.popularity, reverse=True)
        index = min(random.randint(0, 5), len(self.pdt_list) -1)
        chosen_pdt = copy.deepcopy(self.pdt_list[index])
        return chosen_pdt
    
    def delete_pdt(self, product_index):
        for i in range(len(self.pdt_list)):
            if self.pdt_list[i].index == product_index:
                # print("deleting pdt from list", product_index)
                
                if self.pdt_list[i].qty == 1:
                    self.pdt_list.pop(i)
                else:
                    self.pdt_list[i].qty -= 1
                    
                break
        
        return
    
    # Find Product in Sheleves
    def find_shelf(self, chosen_pdt_index):
        for shelf_index, shelf in enumerate(self.shelf_list):
                # print("Checking shelf", shelf_index)
                if chosen_pdt_index in shelf.pdt_set:
                    chosen_shelf_index = shelf_index
        return chosen_shelf_index
    
    def delete_pdt_from_shelf(self, chosen_pdt_index, shelf_index):
        # Remove product from shelf
        shelf = self.shelf_list[shelf_index]
        for i in range(len(shelf.pdts)):
            if shelf.pdts[i].index == chosen_pdt_index:
                # print("QTY of pdt", shelf.pdts[i].qty)
                    if shelf.pdts[i].qty <= 1:
                        print("ran out of product", chosen_pdt_index)
                        del shelf.pdts[i]
                        shelf.pdt_set.discard(chosen_pdt_index)
                    else:
                        shelf.pdts[i].qty-= 1
                    break
                        
        
    def walk(self, init, goal, cur_size, cur_price):
        # Conduct A*
        delta = [(1,0), (-1,0), (0,1), (0,-1)]
        for d in delta:
            y = goal[0]+d[0]
            x = goal[1]+d[1]
            if x >= 0 and x < len(self.grid) and y >= 0 and y < len(self.grid[0]):
                if goal == self.exit:
                    [y,x] = [goal[0], goal[1]]
                path = self.a_star(init, [y,x])
                # print("init", init)
                # print("goal", [y,x])
                if path == "FAIL":
                    z=0
                    #print("FAILURE TO REACH SHELF by going to ", [y,x])
                else:
                    for cell in path:
                        #print("walked", cur_size)
                        self.shopper_density_grid[cell[0]][cell[1]] += cur_size
                        self.price_density_grid[cell[0]][cell[1]] += cur_price
                    cur_pos = [y,x]
                    print("Visited ", y, x)
                    return cur_pos
        
    # Simulates a person entering shop
    def new_shopper(self):
        print("--Testing New Shopper--")
        max_number_of_products = 3
        max_size = 300
        
        cur_number_of_pdt = 0
        cur_size = random.randint(40, 60)
        cur_price = 0
        
        print("Size of Shopper:", cur_size)
        
        # Random Entrance Square
        cur_pos = random.choice(self.entrance)
        
        if len(self.pdt_list) == 0:
            return -1
        while cur_size < max_size and cur_number_of_pdt < 3 and len(self.pdt_list) > 0:
            
            # Choose what Product to Buy
            chosen_pdt = self.choose_pdt()
            if cur_size + chosen_pdt.size > max_size:
                print("next pdt too heavy")
                break
            
            print("Chosen Product:", chosen_pdt.index)
        
            # Find Shelf Index
            chosen_shelf_index = self.find_shelf(chosen_pdt.index)
            print("Shelf Containing Product:", chosen_shelf_index)
        
            # Find Location of Shelf 
            for y in range(len(self.grid)):
                for x in range(len(self.grid[0])):
                    if self.grid[y][x] == chosen_shelf_index:
                        shelf_location = (y,x)
    
            # Walk from cur_pos to another shelf while tracking the movement of the shopper
            cur_pos = self.walk(cur_pos, shelf_location, cur_size, cur_price)
            self.delete_pdt(chosen_pdt.index)
            self.delete_pdt_from_shelf(chosen_pdt.index, chosen_shelf_index)
            self.value_of_goods_bought += chosen_pdt.discounted_price
            cur_number_of_pdt +=1
            cur_price += chosen_pdt.discounted_price
            cur_size += chosen_pdt.size
        
        # Walk to Counter
        cur_pos = self.walk(cur_pos, random.choice(self.counter), cur_size, cur_price)
        
        # Walk to Exit [cur_price == 0 because they aready paid]
        cur_pos = self.walk(cur_pos, random.choice(self.exit), cur_size, 0)
        print("--Finishing Shopper--")
                
        return copy.deepcopy(self.shopper_density_grid)
        
    # Get adjusted Price Grid
    def get_price_grid(self):
        total_price = 0
        for i in range(len(self.price_density_grid)):
            for j in range(len(self.price_density_grid)):
                total_price += self.price_density_grid[i][j]
        
        for i in range(len(self.price_density_grid)):
            for j in range(len(self.price_density_grid)):
                self.price_density_grid[i][j] *= (self.value_of_goods_bought/total_price)
                self.price_density_grid[i][j] = int(self.price_density_grid[i][j])
        
        return copy.deepcopy(self.price_density_grid)
    
    def get_shopper_grid(self):
        return self.shopper_density_grid

## Calculating Product Damage
### Collision Damage / Self-Drops

In [130]:
def loss(shopper_density, price_density):
    p_collision = (min(shopper_density, 300)) **2 / (300*300)
    return p_collision*price_density

def total_loss(shopper_density_grid, price_density_grid):
    net_loss = 0
    for y in range(len(shopper_density_grid)):
        for x in range(len(shopper_density_grid)):
            net_loss += loss(shopper_density_grid[y][x], price_density_grid[y][x])

    return net_loss


## Simulate Layout in Figure 1


In [131]:
def split_row(word): 
    return [char for char in word]  

In [132]:
fig_1_layout =r'''
000aaaaaaaaaa00000aaaaaaaaaa0000aaaaaaaaaa00000g
00000000000000000000000000000000000000000000000g
00000000000000000000000000000000000000000000000g
000000000000000000000000000000000000000000fff00g
000000000000000000000000000000000000000000fff00g
000ccc00ccc00ccc0000d00d00d00d00d000000000fff00g
000ccc00ccc00ccc0000d00dd0d00dd0dd00000000fff00g
000ccc00ccc00ccc0000dd0dd0dd00d0dd00000000fff00g
000000000000000000000d00d00dd0d00dd000000000000g
000000000000000000000dd0dd0dd0dd0dd000000000000g

000ccc00ccc00ccc00000000000000000000000000fff00g
000ccc00ccc00ccc0000eeeee00eeeee00eeeee000fff00g
000ccc00ccc00ccc0000eeeee00eeeee00eeeee000fff00g
000000000000000000000000000000000000000000fff00g
000000000000000000000000000000000000000000fff00g
000ccc00ccc00ccc0000eeeee00eeeee00eeeee000fff00g
000ccc00ccc00ccc0000eeeee00eeeee00eeeee000fff00g
000ccc00ccc00ccc00000000000000000000000000fff00g
000000000000000000000000000000000000000000fff00g
000000000000000000000000000000000000000000fff00g

000000000000000000000000000000000000000000fff00g
000000000000000000000000000000000000000000fff00g
00hh0hh0hh0hh0hh00000ii00iiiiiiiiii00ii000fff00g
00hh0hh0hh0hh0hh00000ii00iiiiiiiiii00ii00000000g
00hh0hh0hh0hh0hh0000000000000000000000000000000g
00hh0hh0hh0hh0hh00000000000000000000000000fff00g
00hh0hh0hh0hh0hh00000ii00iiiiiiiiii00ii000fff00g
00hh0hh0hh0hh0hh00000ii00iiiiiiiiii00ii000fff00g
00hh0hh0hh0hh0hh00000000000000000000000000fff00g
00hh0hh0hh0hh0hh00000000000000000000000000fff00g

00hh0hh0hh0hh0hh00000ii00iiiiiiiiii00ii000fff00g
00hh0hh0hh0hh0hh00000ii00iiiiiiiiii00ii000fff00g
00hh0hh0hh0hh0hh00000000000000000000000000fff00g
000000000000000000000jj00jjjj00jjj00jjj000fff00g
000000000000000000000jj00jjjj00jjj00jjj000fff00g
00000000000000000000000000000000000000000000000g
000000000000000000000000000000000000000000fff00g
000000000000000000000000000000000000000000fff00g
000000000000000000000000000000000000000000fff00g
000000000000000000000000000000000000000000fff00g

000000000000000000000000000000000000000000fff00g
000000000000000000000000000000000000000000fff00g
000000000000000000000000000000000000000000fff00g
000000000000000000000000000000000000000000fff00g
000000000000000000000000000000000000000000fff00g
000000000000000000000000000000000000000000fff000
000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000

'''
fig_1_layout = fig_1_layout.replace('\n\n', '\n')
fig_1_layout = fig_1_layout.strip("\n")

fig_1_layout = fig_1_layout.split("\n")
for i in range(len(fig_1_layout)):
    fig_1_layout[i] = split_row(fig_1_layout[i])

In [133]:
department_dictionary = {
    "a": "Appliances",
    "c": "Audio",
    "d": "Cameras",
    "e": "CellPhones",
    "f": "Computers & Tablets",
    "g": "TV&Home Theater",
    "h": "Video Gaming",
}
shelf_list = [shelf("")]
shelf_counter = 1
for (letter, department_name) in department_dictionary.items():
    for y in range(len(fig_1_layout)):
        for x in range(len(fig_1_layout)):
            if str(fig_1_layout[y][x]) == letter:
                print("found shelf", shelf_counter, "department:", department_name)
                fig_1_layout[y][x] = str(shelf_counter)
                shelf_counter += 1
                shelf_list.append(shelf(department_name))

for y in range(len(fig_1_layout)):
    for x in range(len(fig_1_layout)):
        if not str(fig_1_layout[y][x]).isdigit():
            fig_1_layout[y][x] = str(shelf_counter)
            shelf_counter +=1
        fig_1_layout[y][x] = int(fig_1_layout[y][x])
        
        
    
max_shelf_index = shelf_counter

found shelf 1 department: Appliances
found shelf 2 department: Appliances
found shelf 3 department: Appliances
found shelf 4 department: Appliances
found shelf 5 department: Appliances
found shelf 6 department: Appliances
found shelf 7 department: Appliances
found shelf 8 department: Appliances
found shelf 9 department: Appliances
found shelf 10 department: Appliances
found shelf 11 department: Appliances
found shelf 12 department: Appliances
found shelf 13 department: Appliances
found shelf 14 department: Appliances
found shelf 15 department: Appliances
found shelf 16 department: Appliances
found shelf 17 department: Appliances
found shelf 18 department: Appliances
found shelf 19 department: Appliances
found shelf 20 department: Appliances
found shelf 21 department: Appliances
found shelf 22 department: Appliances
found shelf 23 department: Appliances
found shelf 24 department: Appliances
found shelf 25 department: Appliances
found shelf 26 department: Appliances
found shelf 27 depart

### Process Objects Based on Department to Place into Shelves

In [134]:
pdt_index = 0
pdt_list.sort(key=lambda x: x.department)

for shelf_index in range(1, max_shelf_index+1):
    # print(shelf_index)
    cur_shelf = shelf_list[shelf_index]
    for i in range(3):
        next_pdt = copy.deepcopy(pdt_list[pdt_index])
        print("Department", next_pdt.department)
        if cur_shelf.add_pdt(next_pdt) != -1:
            pdt_index += 1
            if pdt_index == len(pdt_list):
                break
        else:
            break
    print("Products in shelf" , shelf_index, ":", cur_shelf.pdt_set)
    shelf_list.append(cur_shelf)
    if pdt_index == len(pdt_list):
        print("--- Finished All Products with", shelf_index, "out of", max_shelf_index, "shelves --- ")
        break

Department Appliances
product_department Appliances
shelf_department Appliances
Department Appliances
product_department Appliances
shelf_department Appliances
Department Appliances
product_department Appliances
shelf_department Appliances
Products in shelf 1 : {72, 73, 74}
Department Appliances
product_department Appliances
shelf_department Appliances
Department Appliances
product_department Appliances
shelf_department Appliances
Department Appliances
product_department Appliances
shelf_department Appliances
Products in shelf 2 : {75, 76}
Department Appliances
product_department Appliances
shelf_department Appliances
Department Appliances
product_department Appliances
shelf_department Appliances
Department Appliances
product_department Appliances
shelf_department Appliances
Products in shelf 3 : {97, 98, 77}
Department Appliances
product_department Appliances
shelf_department Appliances
Department Appliances
product_department Appliances
shelf_department Appliances
Department Applianc

shelf_department CellPhones
Products in shelf 203 : set()
Department Cell Phones
product_department Cell Phones
shelf_department CellPhones
Products in shelf 204 : set()
Department Cell Phones
product_department Cell Phones
shelf_department CellPhones
Products in shelf 205 : set()
Department Cell Phones
product_department Cell Phones
shelf_department CellPhones
Products in shelf 206 : set()
Department Cell Phones
product_department Cell Phones
shelf_department CellPhones
Products in shelf 207 : set()
Department Cell Phones
product_department Cell Phones
shelf_department CellPhones
Products in shelf 208 : set()
Department Cell Phones
product_department Cell Phones
shelf_department CellPhones
Products in shelf 209 : set()
Department Cell Phones
product_department Cell Phones
shelf_department CellPhones
Products in shelf 210 : set()
Department Cell Phones
product_department Cell Phones
shelf_department Computers & Tablets
Products in shelf 211 : set()
Department Cell Phones
product_depart

Products in shelf 491 : {120, 118, 119}
Department Cell Phones
product_department Cell Phones
shelf_department Appliances
Products in shelf 492 : {121, 122, 125}
Department Cell Phones
product_department Cell Phones
shelf_department Appliances
Products in shelf 493 : {128, 126, 127}
Department Cell Phones
product_department Cell Phones
shelf_department Appliances
Products in shelf 494 : {129, 130}
Department Cell Phones
product_department Cell Phones
shelf_department Appliances
Products in shelf 495 : {131}
Department Cell Phones
product_department Cell Phones
shelf_department Appliances
Products in shelf 496 : set()
Department Cell Phones
product_department Cell Phones
shelf_department Appliances
Products in shelf 497 : set()
Department Cell Phones
product_department Cell Phones
shelf_department Appliances
Products in shelf 498 : set()
Department Cell Phones
product_department Cell Phones
shelf_department Appliances
Products in shelf 499 : set()
Department Cell Phones
product_departme

In [135]:
## Test Layout with Model
import gc
counters = [(40, 6),(40, 7),(40, 8),(40, 9),(40, 10)]
entrances = [[47,30],[47,29],[47,28]]
exit = [[47,35],[47,36],[47,37]]
fig_1_layout_object = layout(fig_1_layout, counters, entrances, exit, shelf_list, pdt_list)
print("Number of products in layout:", len(fig_1_layout_object.pdt_list))

Number of products in layout: 134


In [136]:
shopper_density_grid = fig_1_layout_object.new_shopper()

--Testing New Shopper--
Size of Shopper: 44
Chosen Product: 113
Shelf Containing Product: 965


UnboundLocalError: local variable 'shelf_location' referenced before assignment

In [ ]:
gc.collect()
for i in range(500):
    try:
        shopper_density_grid = fig_1_layout_object.new_shopper()
        if shopper_density_grid == -1:
            print("Exhausted Shop")
            break
    except:
        pass
price_density_grid = fig_1_layout_object.get_price_grid()
shopper_density_grid = fig_1_layout_object.get_shopper_grid()

In [ ]:
plt.imshow(price_density_grid)

In [ ]:
plt.imshow(shopper_density_grid)

In [ ]:
total=0
for pdt in pdt_list:
    total += pdt.discounted_price*pdt.qty
    
print("No of products:", len(pdt_list))
print("Number of products in layout:", len(fig_1_layout_object.pdt_list))
print("Total Value", total)
print("Value of Goods bought",fig_1_layout_object.value_of_goods_bought )
print("Total Loss:", total_loss(shopper_density_grid, price_density_grid))

In [ ]:
for pdt in pdt_list:
    if pdt.index ==  101:
        print(pdt.department)